# Tutorial 8: Cleaning the Archives
## Handling Messy Data

---

### 📺 Video Walkthrough

Watch the video tutorial for this notebook: [YouTube Link](#)

---

### The State of the Records

*"The problem with Quarry records," Chief Archivist Mink said, "is that they're written by trappers. Half of them can't spell. The other half lie. And the ones who can spell and tell the truth tend to get eaten before they finish their reports."*

*She dropped a stack of papers on the apprentice's desk. "These are the raw field reports from the last expedition. Before you can analyze them, you need to clean them."*

*"Clean them?"*

*"Fix the spelling. Fill in the blanks where you can. Mark the inconsistencies. Remove the duplicates—some crews report the same catch twice to inflate their numbers." She paused. "And watch for the impossible. If someone reports catching a Grimslew Fish alive, they're either lying or about to discover their error."*

*—From the training of an apprentice archivist*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Identify and handle missing values
2. Find and remove duplicate records
3. Fix inconsistent text (case, whitespace, typos)
4. Validate data against known rules
5. Convert data types appropriately

By the end, you will be able to transform messy real-world data into clean, analyzable datasets.

---

In [ ]:
import pandas as pd
import numpy as np

# For this tutorial, we'll create a messy version of the catch data
# In real life, you'd receive data in this state

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/yeller-quarry-data-science/main/data/"

# Load clean data to reference
clean_catches = pd.read_csv(BASE_URL + "catches.csv")
creatures = pd.read_csv(BASE_URL + "creatures.csv")

print("Reference data loaded.")

In [ ]:
# Create a messy version for practice
messy_catches = clean_catches.copy()

# Introduce various problems:

# 1. Missing values
messy_catches.loc[3, 'quantity'] = np.nan
messy_catches.loc[7, 'price_per_unit'] = np.nan
messy_catches.loc[12, 'sector'] = np.nan
messy_catches.loc[15, 'condition'] = np.nan
messy_catches.loc[20, 'crew_id'] = np.nan

# 2. Duplicate rows
dup_row = messy_catches.iloc[5].copy()
dup_row['catch_id'] = 'CAT0051'  # Different ID but same data
messy_catches = pd.concat([messy_catches, pd.DataFrame([dup_row])], ignore_index=True)

# Exact duplicate
messy_catches = pd.concat([messy_catches, messy_catches.iloc[[10]]], ignore_index=True)

# 3. Inconsistent text
messy_catches.loc[2, 'sector'] = 'western marsh'  # lowercase
messy_catches.loc[8, 'sector'] = 'Western  Marsh'  # extra space
messy_catches.loc[14, 'sector'] = ' Western Marsh'  # leading space
messy_catches.loc[18, 'condition'] = 'Live'  # capitalized
messy_catches.loc[22, 'condition'] = 'DEAD'  # all caps

# 4. Invalid values
messy_catches.loc[25, 'quantity'] = -5  # negative quantity
messy_catches.loc[28, 'price_per_unit'] = 0  # zero price (might be valid, might not)

# 5. Wrong data types (strings where numbers should be)
messy_catches['quantity'] = messy_catches['quantity'].astype(str)
messy_catches.loc[30, 'quantity'] = 'three'  # text instead of number

print(f"Messy dataset created with {len(messy_catches)} rows")
print("Problems introduced: missing values, duplicates, inconsistent text, invalid values, wrong types")

In [ ]:
# Let's look at the messy data
messy_catches.head(15)

---

## Part 1: Identifying Missing Values

Missing values in pandas are represented as `NaN` (Not a Number) or `None`.

In [ ]:
# Check for missing values in each column
print("Missing values per column:")
print(messy_catches.isnull().sum())

In [ ]:
# Total missing values
total_missing = messy_catches.isnull().sum().sum()
print(f"Total missing values: {total_missing}")

In [ ]:
# Which rows have missing values?
rows_with_missing = messy_catches[messy_catches.isnull().any(axis=1)]
print(f"Rows with missing values: {len(rows_with_missing)}")
rows_with_missing

### Handling Missing Values

You have several options:
1. **Drop** rows or columns with missing values
2. **Fill** with a specific value (0, mean, median, mode)
3. **Forward/backward fill** (use previous or next value)
4. **Leave as is** (some analyses handle NaN automatically)

In [ ]:
# Option 1: Drop rows with any missing values
cleaned_v1 = messy_catches.dropna()
print(f"After dropna: {len(cleaned_v1)} rows (lost {len(messy_catches) - len(cleaned_v1)})")

In [ ]:
# Option 2: Fill missing values
cleaned_v2 = messy_catches.copy()

# Fill missing sector with 'Unknown'
cleaned_v2['sector'] = cleaned_v2['sector'].fillna('Unknown')

# Fill missing condition with the mode (most common value)
cleaned_v2['condition'] = cleaned_v2['condition'].fillna(clean_catches['condition'].mode()[0])

print("After filling sector and condition:")
print(cleaned_v2.isnull().sum())

---

## Part 2: Finding and Removing Duplicates

In [ ]:
# Check for duplicate rows (all columns identical)
duplicates = messy_catches.duplicated()
print(f"Exact duplicate rows: {duplicates.sum()}")

In [ ]:
# View the duplicates
messy_catches[duplicates]

In [ ]:
# Check for duplicates based on specific columns
# (same trap, creature, date, quantity - probably same catch reported twice)
subset_cols = ['trap_id', 'creature_id', 'date', 'quantity']
potential_dups = messy_catches.duplicated(subset=subset_cols, keep=False)
print(f"Potential duplicate catches: {potential_dups.sum()}")
messy_catches[potential_dups]

In [ ]:
# Remove duplicates - keep first occurrence
deduped = messy_catches.drop_duplicates()
print(f"After removing exact duplicates: {len(deduped)} rows")

# Remove based on subset - keep first
deduped_v2 = messy_catches.drop_duplicates(subset=subset_cols, keep='first')
print(f"After removing catch duplicates: {len(deduped_v2)} rows")

---

## Part 3: Fixing Inconsistent Text

Text data often has inconsistencies: different cases, extra spaces, typos.

In [ ]:
# Look at unique values in sector column
print("Unique sector values (before cleaning):")
print(messy_catches['sector'].unique())

In [ ]:
# Count variations
messy_catches['sector'].value_counts()

We can see "Western Marsh" appears in multiple forms. Let's standardize.

In [ ]:
# String cleaning chain
cleaned = messy_catches.copy()

# For string columns, apply cleaning operations
cleaned['sector'] = (
    cleaned['sector']
    .str.strip()           # Remove leading/trailing whitespace
    .str.lower()           # Convert to lowercase
    .str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with single
    .str.title()           # Convert to Title Case
)

print("After cleaning sector:")
print(cleaned['sector'].value_counts())

In [ ]:
# Clean condition column too
cleaned['condition'] = cleaned['condition'].str.strip().str.lower()
print("After cleaning condition:")
print(cleaned['condition'].value_counts())

---

## Part 4: Converting Data Types

In [ ]:
# Check current data types
print(messy_catches.dtypes)

In [ ]:
# Quantity is a string! Let's see what values we have
print("\nQuantity values:")
print(messy_catches['quantity'].unique())

In [ ]:
# Try to convert to numeric - errors='coerce' turns unconvertible values to NaN
cleaned['quantity'] = pd.to_numeric(cleaned['quantity'], errors='coerce')

# See what couldn't be converted
print(f"Values that couldn't be converted: {cleaned['quantity'].isna().sum()}")
print(f"New dtype: {cleaned['quantity'].dtype}")

In [ ]:
# Find the row with 'three'
original_row = messy_catches[messy_catches['quantity'] == 'three']
print("Row with text quantity:")
original_row

In [ ]:
# We could manually fix this if we know the correct value
# For now, let's drop rows where quantity couldn't be converted
cleaned = cleaned.dropna(subset=['quantity'])
print(f"Rows remaining: {len(cleaned)}")

---

## Part 5: Validating Data

In [ ]:
# Check for invalid values

# Negative quantities shouldn't exist
negative_qty = cleaned[cleaned['quantity'] < 0]
print(f"Rows with negative quantity: {len(negative_qty)}")
negative_qty

In [ ]:
# Zero or negative prices might be errors
invalid_price = cleaned[cleaned['price_per_unit'] <= 0]
print(f"Rows with zero/negative price: {len(invalid_price)}")
invalid_price

In [ ]:
# Validate creature_ids against known creatures
valid_creatures = creatures['creature_id'].tolist()
invalid_creatures = cleaned[~cleaned['creature_id'].isin(valid_creatures)]
print(f"Rows with unknown creature_id: {len(invalid_creatures)}")

In [ ]:
# Remove invalid records
validated = cleaned[
    (cleaned['quantity'] > 0) & 
    (cleaned['price_per_unit'] > 0) &
    (cleaned['creature_id'].isin(valid_creatures))
]

print(f"After validation: {len(validated)} rows (removed {len(cleaned) - len(validated)})")

---

## Part 6: Creating a Cleaning Pipeline

In practice, you'll want to combine all cleaning steps into a reusable function.

In [ ]:
def clean_catch_data(df, creatures_ref):
    """
    Clean catch data from the Yeller Quarry Archives.
    
    Steps:
    1. Remove exact duplicates
    2. Convert data types
    3. Standardize text columns
    4. Handle missing values
    5. Validate against known constraints
    
    Returns:
        Cleaned DataFrame
    """
    cleaned = df.copy()
    initial_rows = len(cleaned)
    
    # Step 1: Remove exact duplicates
    cleaned = cleaned.drop_duplicates()
    print(f"After removing duplicates: {len(cleaned)} rows")
    
    # Step 2: Convert quantity to numeric
    cleaned['quantity'] = pd.to_numeric(cleaned['quantity'], errors='coerce')
    
    # Step 3: Standardize text columns
    text_cols = ['sector', 'condition', 'destination']
    for col in text_cols:
        if col in cleaned.columns:
            cleaned[col] = (
                cleaned[col]
                .str.strip()
                .str.lower()
                .str.replace(r'\s+', ' ', regex=True)
                .str.title()
            )
    
    # Condition should be lowercase
    cleaned['condition'] = cleaned['condition'].str.lower()
    
    # Step 4: Handle missing values
    # Drop rows missing critical fields
    critical_cols = ['creature_id', 'quantity', 'date']
    cleaned = cleaned.dropna(subset=critical_cols)
    print(f"After dropping rows with critical missing values: {len(cleaned)} rows")
    
    # Fill non-critical missing values
    cleaned['sector'] = cleaned['sector'].fillna('Unknown')
    cleaned['condition'] = cleaned['condition'].fillna('unknown')
    
    # Step 5: Validate
    valid_creatures = creatures_ref['creature_id'].tolist()
    cleaned = cleaned[
        (cleaned['quantity'] > 0) & 
        (cleaned['creature_id'].isin(valid_creatures))
    ]
    print(f"After validation: {len(cleaned)} rows")
    
    # Summary
    print(f"\nCleaning complete: {initial_rows} → {len(cleaned)} rows")
    print(f"Rows removed: {initial_rows - len(cleaned)}")
    
    return cleaned

In [ ]:
# Apply the cleaning pipeline
final_cleaned = clean_catch_data(messy_catches, creatures)

In [ ]:
# Verify the results
print("\nFinal data quality check:")
print(f"  Missing values: {final_cleaned.isnull().sum().sum()}")
print(f"  Duplicate rows: {final_cleaned.duplicated().sum()}")
print(f"  Unique sectors: {final_cleaned['sector'].nunique()}")
print(f"  Unique conditions: {final_cleaned['condition'].nunique()}")
print(f"  Quantity range: {final_cleaned['quantity'].min()} to {final_cleaned['quantity'].max()}")

---

## ✏️ Exercises

### Exercise 1: Identify All Problems

Before cleaning, write code to identify and count ALL the problems in the messy data:
- Number of missing values per column
- Number of duplicate rows
- Number of unique variations of 'sector' and 'condition'
- Number of invalid quantities (negative or non-numeric)

In [ ]:
# Your code here



### Exercise 2: Custom Mapping

Sometimes you need to map incorrect values to correct ones. Create a mapping dictionary to fix common sector misspellings, then apply it.

In [ ]:
# Your code here
# Hint: Use df['column'].replace(mapping_dict)



### Exercise 3: Date Validation

Check that all dates in the catches are:
1. Valid dates (can be parsed)
2. Within the expected range (1855)
3. Not in the future relative to the dataset's time period

In [ ]:
# Your code here



### Exercise 4: Outlier Detection

Find potential outliers in the price_per_unit column using the IQR method (values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR).

In [ ]:
# Your code here



### Exercise 5: Build Your Own Cleaner

Write a function to clean the incidents data. Consider:
- What columns are critical?
- What values should be validated?
- What text needs standardization?

In [ ]:
# Your code here



---

## 📋 Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Check for missing | `df.isnull().sum()` |
| Drop missing rows | `df.dropna()` |
| Fill missing values | `df.fillna(value)` |
| Find duplicates | `df.duplicated()` |
| Remove duplicates | `df.drop_duplicates()` |
| Strip whitespace | `df['col'].str.strip()` |
| Change case | `df['col'].str.lower()` |
| Replace text | `df['col'].str.replace()` |
| Convert to numeric | `pd.to_numeric(df['col'], errors='coerce')` |
| Validate values | `df[df['col'].isin(valid_list)]` |

---

## ➡️ Next Tutorial

In **Tutorial 9: Seeing the Quarry**, you will learn to visualize your data using matplotlib and seaborn—creating charts that reveal patterns no table can show.

*"A chart is worth a thousand rows," Chief Archivist Mink said. "The senators don't read tables. They look at pictures. Show them a line going up, and they'll fund your expedition. Show them a line going down, and they'll ask why you're still employed."*

---

## 💾 Save Your Work

**File > Save a copy in Drive** to keep your completed notebook.